Re-implement preprocessing in Seurat

> Dimensionality reduction and clus-tering for each dataset was performed by broadly following a modified version of a previously published approach. Using the unnormalized counts, highly varia-ble genes were identified as previously described, by finding outliers with high coefficients of variations as a function of mean expression. Then, within each dataset, depth-normalized counts values were further z-normalized per gene, to yield z-normalized values. The z-normalized values of variable genes per dataset were used as input for principal component analysis. When computing principal components for the stage-5 datasets, we identified genes correlated with cell-cycle marker TOP2A (Pearson correlation greater 0.15), and excluded them. Clustering was carried out using Leiden community detection, a recently published improve-ment on Louvain community detection. 

See [here](https://satijalab.org/seurat/v3.1/pbmc3k_tutorial.html) for standard workflow

In [1]:
library(Seurat)

In [29]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS/LAPACK: /data/gl/g5/yhtgrace/env/miniconda3/envs/r3.6.2/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] Seurat_3.1.5

loaded via a namespace (and not attached):
 [1] nlme_3.1-145        tsne_0.1-3          bitops_1.0-6       
 [4] RcppAnnoy_0.0.16    RColorBrewer_1.1-2  httr_1.4.1         
 [7] repr_1.1.0          sctransform_0.2.1   tools_3.6.2        
[10] R6_2.4.1            irlba_2

Load data

In [16]:
meta.data <- read.table("../data/Veres2019/GSE114412_Stage_5.all.cell_metadata.tsv.gz", header = T, row.names = 1)
head(meta.data)

,Assigned_cluster,Assigned_subcluster,tSNE_dim1,tSNE_dim2,Differentiation,CellWeek,Lib_prep_batch,Indrops_barcode_sequence
,<fct>,<fct>,<dbl>,<dbl>,<int>,<int>,<fct>,<fct>
stg5diff1_S5d0_b1.bcEKBI,prog_sox2,prog_sox2,-23.483145,-17.51581431,1,0,stg5diff1_S5d0_b1,AGGCAACG-AAGCGTAC
stg5diff1_S5d0_b1.bcDAND,fev_high_isl_low,fev_high_isl_low__day0,6.776112,29.83986815,1,0,stg5diff1_S5d0_b1,GAATGGAAAT-AGCGAAGT
stg5diff1_S5d0_b1.bcGQEK,prog_sox2,prog_sox2,-27.248994,-14.21536773,1,0,stg5diff1_S5d0_b1,TGAGGTCTGAC-TCTCACTT
stg5diff1_S5d0_b1.bcFOEC,prog_sox2,prog_sox2,-30.845154,-7.47394697,1,0,stg5diff1_S5d0_b1,ACTGAGTGC-AAGCGTAC
stg5diff1_S5d0_b1.bcAVAX,prog_sox2,prog_sox2,-7.391701,-16.01483610,1,0,stg5diff1_S5d0_b1,AGATGTATT-ATGACTTT
stg5diff1_S5d0_b1.bcFEDZ,prog_nkx61,prog_nkx61,-28.494072,-0.05024911,1,0,stg5diff1_S5d0_b1,TTTGTGTC-GTCGTCGT


In [17]:
dim(meta.data)

[1] 51274     8

In [18]:
raw.cts <- read.table("../data/Veres2019/GSE114412_Stage_5.all.processed_counts.tsv.gz", 
    sep = "\t", header = T, comment.char = "", row.names = 1)

In [23]:
dim(raw.cts)

[1] 51274 16224

Create Seurat object

Note that a gene is valid if it is observed in at least 10 cells. Cell-cycle genes are also excluded from the PCA calculation, but we might actually expect that to contain useful information for estimating cell proliferation rates. For now, retain all genes observed in at least 10 cells

In [22]:
sc <- CreateSeuratObject(counts = t(raw.cts), meta.data = meta.data, min.cells = 10)
sc

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


An object of class Seurat 
16066 features across 51274 samples within 1 assay 
Active assay: RNA (16066 features, 0 variable features)

Normalize data

In [24]:
sc <- NormalizeData(object = sc)
sc

An object of class Seurat 
16066 features across 51274 samples within 1 assay 
Active assay: RNA (16066 features, 0 variable features)

Select top 2500 highly variable genes

In [30]:
sc <- FindVariableFeatures(object = sc, nfeatures = 2500)
sc

An object of class Seurat 
16066 features across 51274 samples within 1 assay 
Active assay: RNA (16066 features, 2500 variable features)

Scale the data for these features

In [32]:
sc <- ScaleData(sc, features = VariableFeatures(sc))
sc

Centering and scaling data matrix



An object of class Seurat 
16066 features across 51274 samples within 1 assay 
Active assay: RNA (16066 features, 2500 variable features)

Hand off downstream preprocessing to python 

Save normalized, unscaled data for top 2500 highly variable genes

In [43]:
write.csv(t(as.matrix(sc[['RNA']]@data[VariableFeatures(sc),])), file = "../data/Veres2019/Stage_5.Seurat.csv")